In [1]:
%load_ext autoreload
%load_ext autotime

time: 821 µs (started: 2023-01-04 21:02:42 -08:00)


In [2]:
import sys
sys.path.append("../")

import warnings
warnings.filterwarnings('ignore')

time: 992 µs (started: 2023-01-04 21:02:42 -08:00)


In [3]:
%autoreload 2

from vani.recorder import RecorderAnalyzer

log_dir = "/p/vast1/iopp/recorder_app_logs/cm1/nodes-32/workflow-4/_parquet"
    # log_dir = "/p/gpfs1/iopp/recorder_app_logs/hacc/nodes-32/workflow-0/_parquet"
    # log_dir = "/p/gpfs1/iopp/recorder_app_logs/montage/nodes-32/_parquet"
    # log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
    # log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-jag/nodes-32/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
        force_local=True,
        cluster_settings=dict(
            cores=8,
            dashboard_port=3646,
            local_directory="/var/tmp/dask-recorder",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

time: 8.86 s (started: 2023-01-04 21:02:43 -08:00)


In [4]:
recorder_analyzer.cluster_manager.clients

{'local': <Client: 'tcp://192.168.131.2:33827' processes=8 threads=40>,
 'trange': <Client: 'tcp://192.168.131.2:32889' processes=8 threads=40>,
 'file_id': <Client: 'tcp://192.168.131.2:42165' processes=8 threads=40>,
 'proc_id': <Client: 'tcp://192.168.131.2:38016' processes=8 threads=40>}

time: 3.67 ms (started: 2023-01-04 21:02:52 -08:00)


In [5]:
res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

hlm trange 27
hlm file_id 41
hlm proc_id 32
perm ('trange', 'file_id') 40
perm ('trange', 'proc_id') 32
perm ('file_id', 'trange') 26
perm ('file_id', 'proc_id') 32
perm ('proc_id', 'trange') 27
perm ('proc_id', 'file_id') 40
llc ('trange', 'file_id') 40 80
llc ('trange', 'proc_id') 32 65
llc ('file_id', 'trange') 26 51
llc ('file_id', 'proc_id') 32 65
llc ('proc_id', 'trange') 27 53
llc ('proc_id', 'file_id') 40 80
llc-alt ('trange', 'file_id', 'proc_id') 40 1280
llc-alt ('trange', 'proc_id', 'file_id') 32 775
llc-alt ('file_id', 'trange', 'proc_id') 26 1278
llc-alt ('file_id', 'proc_id', 'trange') 32 392
llc-alt ('proc_id', 'trange', 'file_id') 27 85
llc-alt ('proc_id', 'file_id', 'trange') 40 392
time: 8.02 s (started: 2023-01-04 21:02:52 -08:00)


In [6]:
llc, llc_at = res

time: 792 µs (started: 2023-01-04 21:03:00 -08:00)


In [7]:
llc[list(llc.keys())[0]]

acc_pat         app        bw            \
                               max min nunique       fmt       per   
file_id             io_cat                                           
4531734247156679037 3            0   0       1  0.00GB/s  0.000000   
4531734247515089012 3            0   0       1  0.00GB/s  0.000000   
4531734246187791667 3            0   0       1  0.00GB/s  0.000000   
4531734247569105940 3            0   0       1  0.00GB/s  0.000000   
4531734247572069952 3            0   0       1  0.00GB/s  0.000000   
...                            ...  ..     ...       ...       ...   
4531734246726558288 2            0   0       1  0.45GB/s  0.000689   
4531734244818756329 2            0   0       1  0.48GB/s  0.000731   
4531734246458134144 2            0   0       1  0.53GB/s  0.000802   
4531734248478099672 2            0   0       1  0.66GB/s  0.001007   
4531734247415627620 2            0   0       1  0.71GB/s  0.001074   

                                              duration            file_id  \
                                     sum           per        sum nunique   
file_id             io_cat                                                  
4531734247156679037 3       0.000000e+00  6.740286e-01  64.806252       1   
4531734247515089012 3       0.000000e+00  6.037203e-02   5.804627       1   
4531734246187791667 3       0.000000e+00  4.378893e-02   4.210202       1   
4531734247569105940 3       0.000000e+00  4.276112e-02   4.111381       1   
4531734247572069952 3       0.000000e+00  3.946026e-02   3.794011       1   
...                                  ...           ...        ...     ...   
4531734246726558288 2       4.880317e+08  1.051508e-06   0.000101       1   
4531734244818756329 2       5.174972e+08  9.475013e-07   0.000091       1   
4531734246458134144 2       5.680000e+08  8.632558e-07   0.000083       1   
4531734248478099672 2       7.132224e+08  6.874845e-07   0.000066       1   
4531734247415627620 2       7.603871e+08  6.448417e-07   0.000062       1   

                           hostname  ... proc_id  rank                size  \
                            nunique  ... nunique   max min nunique     fmt   
file_id             io_cat           ...                                     
4531734247156679037 3             1  ...      32  1240   0      32  0.00GB   
4531734247515089012 3             1  ...      32  1240   0      32  0.00GB   
4531734246187791667 3             1  ...      32  1240   0      32  0.00GB   
4531734247569105940 3             1  ...      32  1240   0      32  0.00GB   
4531734247572069952 3             1  ...      32  1240   0      32  0.00GB   
...                             ...  ...     ...   ...  ..     ...     ...   
4531734246726558288 2             1  ...       1     0   0       1  0.00GB   
4531734244818756329 2             1  ...       1     0   0       1  0.00GB   
4531734246458134144 2             1  ...       1     0   0       1  0.00GB   
4531734248478099672 2             1  ...       1     0   0       1  0.00GB   
4531734247415627620 2             1  ...       1     0   0       1  0.00GB   

                                              xfer                
                                 per    sum    max   mean    min  
file_id             io_cat                                        
4531734247156679037 3       0.000000      0   <4KB   <4KB   <4KB  
4531734247515089012 3       0.000000      0   <4KB   <4KB   <4KB  
4531734246187791667 3       0.000000      0   <4KB   <4KB   <4KB  
4531734247569105940 3       0.000000      0   <4KB   <4KB   <4KB  
4531734247572069952 3       0.000000      0   <4KB   <4KB   <4KB  
...                              ...    ...    ...    ...    ...  
4531734246726558288 2       0.000002  49340  ~64KB  ~64KB  ~64KB  
4531734244818756329 2       0.000002  47144  ~64KB  ~64KB  ~64KB  
4531734246458134144 2       0.000002  47144  ~64KB  ~64KB  ~64KB  
4531734248478099672 2       0.000002  47144  ~64KB  ~64KB  ~64KB  
453

time: 66.5 ms (started: 2023-01-04 21:03:01 -08:00)


In [8]:
llc[list(llc.keys())[1]]

acc_pat         app           bw                \
                               max min nunique          fmt           per   
proc_id             io_cat                                                  
6860131432589748884 3            1   0       1     0.00GB/s  1.101769e-07   
6860131432629070484 3            0   0       1     0.00GB/s  0.000000e+00   
6860131432615963284 3            0   0       1     0.00GB/s  0.000000e+00   
6860131432636934804 3            0   0       1     0.00GB/s  0.000000e+00   
6860131432613341844 3            0   0       1     0.00GB/s  0.000000e+00   
...                            ...  ..     ...          ...           ...   
6860131432610720404 1            0   0       1  1775.57GB/s  4.200312e-02   
6860131432634313364 1            0   0       1  1929.01GB/s  4.563302e-02   
6860131432618584724 1            0   0       1  1929.01GB/s  4.563302e-02   
6860131432668392084 1            0   0       1  1977.85GB/s  4.678828e-02   
6860131432647420564 1            0   0       1  2003.21GB/s  4.738813e-02   

                                              duration           file_id  \
                                     sum           per       sum nunique   
proc_id             io_cat                                                 
6860131432589748884 3       5.000877e+06  3.642998e-02  3.534376     774   
6860131432629070484 3       0.000000e+00  3.548329e-02  3.442531      38   
6860131432615963284 3       0.000000e+00  3.434119e-02  3.331725      38   
6860131432636934804 3       0.000000e+00  3.386656e-02  3.285678      38   
6860131432613341844 3       0.000000e+00  3.356776e-02  3.256689      38   
...                                  ...           ...       ...     ...   
6860131432610720404 1       1.906502e+12  9.070449e-08  0.000009       1   
6860131432634313364 1       2.071261e+12  8.348936e-08  0.000008       1   
6860131432618584724 1       2.071261e+12  8.348936e-08  0.000008       1   
6860131432668392084 1       2.123698e+12  8.142790e-08  0.000008       1   
6860131432647420564 1       2.150925e+12  8.039716e-08  0.000008       1   

                           hostname  ... proc_id  rank                  size  \
                            nunique  ... nunique   max   min nunique     fmt   
proc_id             io_cat           ...                                       
6860131432589748884 3             1  ...       1     0     0       1  0.02GB   
6860131432629070484 3             1  ...       1   600   600       1  0.00GB   
6860131432615963284 3             1  ...       1   400   400       1  0.00GB   
6860131432636934804 3             1  ...       1   720   720       1  0.00GB   
6860131432613341844 3             1  ...       1   360   360       1  0.00GB   
...                             ...  ...     ...   ...   ...     ...     ...   
6860131432610720404 1             1  ...       1   320   320       1  0.02GB   
6860131432634313364 1             1  ...       1   680   680       1  0.02GB   
6860131432618584724 1             1  ...       1   440   440       1  0.02GB   
6860131432668392084 1             1  ...       1  1200  1200       1  0.02GB   
6860131432647420564 1             1  ...       1   880   880       1  0.02GB   

                                                  xfer                
                                 per       sum     max   mean    min  
proc_id             io_cat                                            
6860131432589748884 3       0.009717  17674980  ~256KB   <4KB   <4KB  
6860131432629070484 3       0.000000         0    <4KB   <4KB   <4KB  
6860131432615963284 3       0.000000         0    <4KB   <4KB   <4KB  
6860131432636934804 3       0.000000         0    <4KB   <4KB   <4KB  
6860131432613341844 3       0.000000         0    <4KB   <4KB   <4KB  
...                              ...       ...     ...    ...    ...  
6860131432610720404 1       0.009224  16777216   ~16MB  ~16MB  ~16MB  
6860131432634313364 1       0.009224  16777216   ~16

time: 64 ms (started: 2023-01-04 21:03:01 -08:00)


In [9]:
llc[list(llc.keys())[2]]

acc_pat         app          bw                              \
                  max min nunique         fmt           per           sum   
trange io_cat                                                               
5      3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
450    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
668    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
344    3            1   0       1    0.00GB/s  1.812139e-08  1.177777e+04   
233    3            1   0       1    0.00GB/s  1.321160e-08  8.586712e+03   
128    3            1   0       1    0.00GB/s  6.769039e-09  4.399451e+03   
561    3            1   0       1    0.00GB/s  5.127312e-08  3.332431e+04   
129    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
345    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
449    3            1   0       1    0.00GB/s  1.812354e-07  1.177916e+05   
667    3            1   0       1    0.00GB/s  4.311754e-07  2.802369e+05   
634    3            1   0       1    0.00GB/s  4.377857e-07  2.845332e+05   
3      3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
7      3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
3      1            0   0       1  542.61GB/s  8.964369e-01  5.826277e+11   
185    3            1   0       1    0.00GB/s  1.165232e-06  7.573276e+05   
150    3            1   0       1    0.00GB/s  9.476529e-07  6.159149e+05   
345    2            0   0       1    4.74GB/s  7.828355e-03  5.087939e+09   
361    3            1   0       1    0.00GB/s  2.605299e-06  1.693281e+06   
613    3            1   0       1    0.00GB/s  4.922954e-06  3.199611e+06   
668    2            0   0       1    4.93GB/s  8.152407e-03  5.298553e+09   
129    2            0   0       1    4.44GB/s  7.340947e-03  4.771154e+09   
408    3            1   0       1    0.00GB/s  3.368602e-06  2.189380e+06   
450    2            0   0       1    5.09GB/s  8.405230e-03  5.462872e+09   
60     3            1   0       1    0.00GB/s  3.572926e-07  2.322178e+05   
562    2            0   0       1    4.51GB/s  7.444896e-03  4.838715e+09   
199    3            1   0       1    0.00GB/s  1.801543e-06  1.170890e+06   
454    3            1   0       1    0.00GB/s  4.668735e-06  3.034385e+06   
234    2            0   0       1    5.37GB/s  8.871823e-03  5.766128e+09   
6      3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
344    2            0   0       1    3.44GB/s  5.676921e-03  3.689642e+09   
7      2            0   0       1    6.15GB/s  1.016554e-02  6.606961e+09   
669    2            0   0       1    4.49GB/s  7.411623e-03  4.817089e+09   
234    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
233    2            0   0       1    4.29GB/s  7.088631e-03  4.607165e+09   
6      2            0   0       1    5.17GB/s  8.539581e-03  5.550191e+09   
562    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
561    2            0   0       1    5.25GB/s  8.667439e-03  5.633291e+09   
128    2            0   0       1    2.69GB/s  4.439017e-03  2.885083e+09   
449    2            0   0       1    0.00GB/s  2.842845e-06  1.847671e+06   
634    2            0   0       1    0.06GB/s  9.444025e-05  6.138024e+07   
669    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
60     2            0   0       1    0.24GB/s  3.977211e-04  2.584938e+08   
454    2            0   0       1    0.27GB/s  4.445689e-04  2.889419e+08   
408    2            0   0       1    0.28GB/s  4.548608e-04  2.956310e+08   
185    2            0   0       1    0.28GB/s  4.704499e-04  3.057629e+08   
361    2            0   0       1    0.29GB/s  4.808341e-04  3.125120e+08   
613    2            0   0       1    0.30GB/s  4.982886e-04  3.238563e+08   
199    2            0   0       1    0.32GB/s  5.318781e-04  3.456874e+08   
667    2  

time: 81.9 ms (started: 2023-01-04 21:03:01 -08:00)


In [10]:
llc[list(llc.keys())[3]]

acc_pat         app           bw                \
                               max min nunique          fmt           per   
proc_id             io_cat                                                  
6860131432589748884 3            1   0       1     0.00GB/s  1.101769e-07   
6860131432629070484 3            0   0       1     0.00GB/s  0.000000e+00   
6860131432615963284 3            0   0       1     0.00GB/s  0.000000e+00   
6860131432636934804 3            0   0       1     0.00GB/s  0.000000e+00   
6860131432613341844 3            0   0       1     0.00GB/s  0.000000e+00   
...                            ...  ..     ...          ...           ...   
6860131432610720404 1            0   0       1  1775.57GB/s  4.200312e-02   
6860131432634313364 1            0   0       1  1929.01GB/s  4.563302e-02   
6860131432618584724 1            0   0       1  1929.01GB/s  4.563302e-02   
6860131432668392084 1            0   0       1  1977.85GB/s  4.678828e-02   
6860131432647420564 1            0   0       1  2003.21GB/s  4.738813e-02   

                                              duration           file_id  \
                                     sum           per       sum nunique   
proc_id             io_cat                                                 
6860131432589748884 3       5.000877e+06  3.642998e-02  3.534376     774   
6860131432629070484 3       0.000000e+00  3.548329e-02  3.442531      38   
6860131432615963284 3       0.000000e+00  3.434119e-02  3.331725      38   
6860131432636934804 3       0.000000e+00  3.386656e-02  3.285678      38   
6860131432613341844 3       0.000000e+00  3.356776e-02  3.256689      38   
...                                  ...           ...       ...     ...   
6860131432610720404 1       1.906502e+12  9.070449e-08  0.000009       1   
6860131432634313364 1       2.071261e+12  8.348936e-08  0.000008       1   
6860131432618584724 1       2.071261e+12  8.348936e-08  0.000008       1   
6860131432668392084 1       2.123698e+12  8.142790e-08  0.000008       1   
6860131432647420564 1       2.150925e+12  8.039716e-08  0.000008       1   

                           hostname  ... proc_id  rank                  size  \
                            nunique  ... nunique   max   min nunique     fmt   
proc_id             io_cat           ...                                       
6860131432589748884 3             1  ...       1     0     0       1  0.02GB   
6860131432629070484 3             1  ...       1   600   600       1  0.00GB   
6860131432615963284 3             1  ...       1   400   400       1  0.00GB   
6860131432636934804 3             1  ...       1   720   720       1  0.00GB   
6860131432613341844 3             1  ...       1   360   360       1  0.00GB   
...                             ...  ...     ...   ...   ...     ...     ...   
6860131432610720404 1             1  ...       1   320   320       1  0.02GB   
6860131432634313364 1             1  ...       1   680   680       1  0.02GB   
6860131432618584724 1             1  ...       1   440   440       1  0.02GB   
6860131432668392084 1             1  ...       1  1200  1200       1  0.02GB   
6860131432647420564 1             1  ...       1   880   880       1  0.02GB   

                                                  xfer                
                                 per       sum     max   mean    min  
proc_id             io_cat                                            
6860131432589748884 3       0.009717  17674980  ~256KB   <4KB   <4KB  
6860131432629070484 3       0.000000         0    <4KB   <4KB   <4KB  
6860131432615963284 3       0.000000         0    <4KB   <4KB   <4KB  
6860131432636934804 3       0.000000         0    <4KB   <4KB   <4KB  
6860131432613341844 3       0.000000         0    <4KB   <4KB   <4KB  
...                              ...       ...     ...    ...    ...  
6860131432610720404 1       0.009224  16777216   ~16MB  ~16MB  ~16MB  
6860131432634313364 1       0.009224  16777216   ~16

time: 58.8 ms (started: 2023-01-04 21:03:02 -08:00)


In [11]:
llc[list(llc.keys())[4]]

acc_pat         app          bw                              \
                  max min nunique         fmt           per           sum   
trange io_cat                                                               
5      3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
450    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
668    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
344    3            1   0       1    0.00GB/s  1.812024e-08  1.177777e+04   
233    3            1   0       1    0.00GB/s  1.321076e-08  8.586712e+03   
128    3            1   0       1    0.00GB/s  6.768610e-09  4.399451e+03   
561    3            1   0       1    0.00GB/s  5.126987e-08  3.332431e+04   
129    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
345    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
449    3            1   0       1    0.00GB/s  1.812239e-07  1.177916e+05   
667    3            1   0       1    0.00GB/s  4.311480e-07  2.802369e+05   
634    3            1   0       1    0.00GB/s  4.377579e-07  2.845332e+05   
3      3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
7      3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
3      1            0   0       1  542.61GB/s  8.963800e-01  5.826277e+11   
185    3            1   0       1    0.00GB/s  1.165158e-06  7.573276e+05   
150    3            1   0       1    0.00GB/s  9.475928e-07  6.159149e+05   
345    2            0   0       1    4.74GB/s  7.827858e-03  5.087939e+09   
361    3            1   0       1    0.00GB/s  2.605133e-06  1.693281e+06   
613    3            1   0       1    0.00GB/s  4.922642e-06  3.199611e+06   
668    2            0   0       1    4.93GB/s  8.151890e-03  5.298553e+09   
129    2            0   0       1    4.44GB/s  7.340481e-03  4.771154e+09   
408    3            1   0       1    0.00GB/s  3.368388e-06  2.189380e+06   
450    2            0   0       1    5.09GB/s  8.404697e-03  5.462872e+09   
60     3            1   0       1    0.00GB/s  3.572700e-07  2.322178e+05   
562    2            0   0       1    4.51GB/s  7.444423e-03  4.838715e+09   
199    3            1   0       1    0.00GB/s  1.801428e-06  1.170890e+06   
454    3            1   0       1    0.00GB/s  4.668439e-06  3.034385e+06   
234    2            0   0       1    5.37GB/s  8.871260e-03  5.766128e+09   
6      3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
344    2            0   0       1    3.44GB/s  5.676561e-03  3.689642e+09   
7      2            0   0       1    6.15GB/s  1.016489e-02  6.606961e+09   
669    2            0   0       1    4.49GB/s  7.411152e-03  4.817089e+09   
234    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
233    2            0   0       1    4.29GB/s  7.088182e-03  4.607165e+09   
6      2            0   0       1    5.17GB/s  8.539039e-03  5.550191e+09   
562    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
561    2            0   0       1    5.25GB/s  8.666889e-03  5.633291e+09   
128    2            0   0       1    2.69GB/s  4.438736e-03  2.885083e+09   
556    3            1   0       1    0.01GB/s  2.231128e-05  1.450185e+07   
       2            0   0       1    0.02GB/s  4.112435e-05  2.672994e+07   
449    2            0   0       1    0.00GB/s  2.842665e-06  1.847671e+06   
634    2            0   0       1    0.06GB/s  9.443426e-05  6.138024e+07   
669    3            0   0       1    0.00GB/s  0.000000e+00  0.000000e+00   
60     2            0   0       1    0.24GB/s  3.976959e-04  2.584938e+08   
454    2            0   0       1    0.27GB/s  4.445407e-04  2.889419e+08   
408    2            0   0       1    0.28GB/s  4.548319e-04  2.956310e+08   
185    2            0   0       1    0.28GB/s  4.704201e-04  3.057629e+08   
361    2            0   0       1    0.29GB/s  4.808036e-04  3.125120e+08   
613    2  

time: 77.4 ms (started: 2023-01-04 21:03:02 -08:00)


In [12]:
llc[list(llc.keys())[5]]

acc_pat         app        bw            \
                               max min nunique       fmt       per   
file_id             io_cat                                           
4531734247156679037 3            0   0       1  0.00GB/s  0.000000   
4531734247515089012 3            0   0       1  0.00GB/s  0.000000   
4531734246187791667 3            0   0       1  0.00GB/s  0.000000   
4531734247569105940 3            0   0       1  0.00GB/s  0.000000   
4531734247572069952 3            0   0       1  0.00GB/s  0.000000   
...                            ...  ..     ...       ...       ...   
4531734246726558288 2            0   0       1  0.45GB/s  0.003322   
4531734244818756329 2            0   0       1  0.48GB/s  0.003522   
4531734246458134144 2            0   0       1  0.53GB/s  0.003866   
4531734248478099672 2            0   0       1  0.66GB/s  0.004854   
4531734247415627620 2            0   0       1  0.71GB/s  0.005175   

                                              duration            file_id  \
                                     sum           per        sum nunique   
file_id             io_cat                                                  
4531734247156679037 3       0.000000e+00  6.736804e-01  64.806252       1   
4531734247515089012 3       0.000000e+00  6.034084e-02   5.804627       1   
4531734246187791667 3       0.000000e+00  4.376631e-02   4.210202       1   
4531734247569105940 3       0.000000e+00  4.273903e-02   4.111381       1   
4531734247572069952 3       0.000000e+00  3.943988e-02   3.794011       1   
...                                  ...           ...        ...     ...   
4531734246726558288 2       4.880317e+08  1.050965e-06   0.000101       1   
4531734244818756329 2       5.174972e+08  9.470119e-07   0.000091       1   
4531734246458134144 2       5.680000e+08  8.628099e-07   0.000083       1   
4531734248478099672 2       7.132224e+08  6.871293e-07   0.000066       1   
4531734247415627620 2       7.603871e+08  6.445086e-07   0.000062       1   

                           hostname  ... proc_id  rank                size  \
                            nunique  ... nunique   max min nunique     fmt   
file_id             io_cat           ...                                     
4531734247156679037 3             1  ...      32  1240   0      32  0.00GB   
4531734247515089012 3             1  ...      32  1240   0      32  0.00GB   
4531734246187791667 3             1  ...      32  1240   0      32  0.00GB   
4531734247569105940 3             1  ...      32  1240   0      32  0.00GB   
4531734247572069952 3             1  ...      32  1240   0      32  0.00GB   
...                             ...  ...     ...   ...  ..     ...     ...   
4531734246726558288 2             1  ...       1     0   0       1  0.00GB   
4531734244818756329 2             1  ...       1     0   0       1  0.00GB   
4531734246458134144 2             1  ...       1     0   0       1  0.00GB   
4531734248478099672 2             1  ...       1     0   0       1  0.00GB   
4531734247415627620 2             1  ...       1     0   0       1  0.00GB   

                                              xfer                
                                 per    sum    max   mean    min  
file_id             io_cat                                        
4531734247156679037 3       0.000000      0   <4KB   <4KB   <4KB  
4531734247515089012 3       0.000000      0   <4KB   <4KB   <4KB  
4531734246187791667 3       0.000000      0   <4KB   <4KB   <4KB  
4531734247569105940 3       0.000000      0   <4KB   <4KB   <4KB  
4531734247572069952 3       0.000000      0   <4KB   <4KB   <4KB  
...                              ...    ...    ...    ...    ...  
4531734246726558288 2       0.000044  49340  ~64KB  ~64KB  ~64KB  
4531734244818756329 2       0.000042  47144  ~64KB  ~64KB  ~64KB  
4531734246458134144 2       0.000042  47144  ~64KB  ~64KB  ~64KB  
4531734248478099672 2       0.000042  47144  ~64KB  ~64KB  ~64KB  
453

time: 60.8 ms (started: 2023-01-04 21:03:02 -08:00)
